In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Reading file and convert to NUMPY array

In [5]:
df = pd.read_csv("*All_info.csv")

In [ ]:
df[["start_date", "end_date"]] = df[["start_date", "end_date"]].apply(pd.to_datetime)

In [ ]:
df['duration'] = df['duration'].apply(lambda i: dt.timedelta(days=i))

In [ ]:
a = df.to_numpy()

In [ ]:
np.shape(a) # each column is a "y"

# Understanding how to process matrix

Instead of calculating the data by looping nested, we can use the transpose (convert rows to columns and vice-versa) to calculate the matrix.

BUT... we have to adjust our data:

- We have to split our data for each column (start_date, end_date, duration and id)
- After splintting the data, we have to make sure the shape is (n x 1), in order to calculate the TRANSPOSE
- Then we can apply our process

In [ ]:
a[:,0].shape # wrong shape

In [ ]:
a[:,1][np.newaxis].shape # right shape

In [ ]:
a[:,1][np.newaxis].T.shape # applying the transpose...

In [ ]:
# Splitting the data
# a0 = a[:150,0][np.newaxis] #start_date
# a1 = a[:150,1][np.newaxis] #end_date
# a2 = a[:150,2][np.newaxis] #duration
# a3 = a[:150,3][np.newaxis] #id

##### UNCOMMENT BELOW TO RUN ALL DATA !!!#####
a0 = a[:,0][np.newaxis] #start_date
a1 = a[:,1][np.newaxis] #end_date
a2 = a[:,2][np.newaxis] #duration
a3 = a[:,3][np.newaxis] #id

## Defining the conditions
> Connectivity between catchments is defined by drought events. 
If a drought event in a given catchment has a temporal overlap of 50% or more with an event in another catchment, these catchments are considered connected.

In [ ]:
cond1 = (a0.T<=a1)&(a0<=a1.T)&(a0!=a0.T)#(a0.T<=a1) means start_date2 <= end_date1
                                        #(a0<=a1.T) means start_date1 <= end_date2
                                        #(a0!=a0.T) to desconsider the diagonal of matrix - ids connected with themselves

In [ ]:
latest_start = np.maximum(a0, a0.T) # this function is element-wise comparison between start_date1 and start_date2
latest_start = np.where(cond1, latest_start, 0) # we can filter the data based on the first condition, if the condition are False the element gets a 0

In [ ]:
# same ideia as the latest_start for the end_date
earlist_end = np.minimum(a1, a1.T)
earlist_end = np.where(cond1, earlist_end, 0)

In [ ]:
delta = earlist_end - latest_start # difference between both (making sure the operation are between Zeros (elements that doesnt meet the conditions) and TIMEDELTA)

In [ ]:
a2_f = np.where(cond1, a2,0) # filtering the duration based on the condition 1

In [ ]:
oc = np.where(delta>=a2_f/2, delta, 0) # filtering the delta, if the values meet the criteria, then oc takes delta... otherwise 0

In [ ]:
occur = oc!=0  # since we do not care about the value itself, but the if the criteria is met or not, we can check if True or False
               # remember 0 == False, 1 == True

In [ ]:
df_ocoor=pd.DataFrame(occur)

## Visual representation of the conditions

In [ ]:
#start1 <= end2
plt.figure(figsize=(10,10),dpi=300)
plt.imshow(a0<=a1.T, cmap="PiYG")
plt.colorbar()
plt.xlabel("drought events")
plt.ylabel("drought events")

In [ ]:
#start2 <= end1
plt.figure(figsize=(10,10),dpi=300)
plt.imshow(a0.T<=a1, cmap="PiYG")
plt.colorbar()
plt.xlabel("drought events")
plt.ylabel("drought events")

In [ ]:
#minimun condition of dates (if minimum overlap occurs) and if delta >= duration/2
plt.figure(figsize=(10,10),dpi=300)
plt.imshow(occur, cmap="PiYG")
plt.xlabel("drought events")
plt.ylabel("drought events")

In [ ]:
#minimum condition of dates (if minimum overlap occurs)
plt.figure(figsize=(10,10),dpi=300)
plt.imshow(cond1, cmap="PiYG" ) 
plt.xlabel("drought events")
plt.ylabel("drought events")

In [ ]:
# points that reach minimum condition but failed in delta >= duration/2
plt.figure(figsize=(10,10),dpi=300)
plt.imshow((cond1-occur.astype(int)), cmap="PiYG" ) 
plt.xlabel("drought events")
plt.ylabel("drought events")

## Save the information

In [ ]:
#quantity of ids that are connected only in one way
np.sum(((a0.T<=a1)&(a0<=a1.T)).astype(int)-occur.astype(int)) 

In [ ]:
# Getting ids where the condition is True
x = np.where(occur)[0] #indices
y = np.where(occur)[1] #indices

id_1=a3[0][x]
id_2=a3[0][y]
datex=a1[0][x]
datey=a1[0][y]

In [ ]:
np.save("id_1.npy", x) #saving x index
np.save("id_2.npy", y) #saving y index

In [ ]:
#transform in dataframe
id1= pd.DataFrame(id_1)
id2= pd.DataFrame(id_2)
date_id1 = pd.DataFrame(datex)
date_id2 = pd.DataFrame(datey)

In [ ]:
#rename the dataframe colums
id1.rename(columns={0:"id1"}, inplace=True)
date_id1.rename(columns={0:"date_id1"}, inplace=True)
date_id2.rename(columns={0:"date_id2"}, inplace=True)
id2.rename(columns={0:"id2"}, inplace=True)

In [ ]:
#join all the information in one dataframe
all_ids=pd.DataFrame()
all_ids["id1"]=id1["id1"]
all_ids["id2"]=id2["id2"]
all_ids["date_id1"]=date_id1["date_id1"]
all_ids["date_id2"]=date_id2["date_id2"]

In [ ]:
#Save the information in a unique file
all_ids.to_csv("*.csv")

In [ ]:
#check number of connections between 2 ids
all_ids.loc[(all_ids["id1"]==5)&
            (all_ids["id2"]==1)]

In [ ]:
#group Id 1 and Id2 in two columns
size_ids=all_ids.groupby(by=["id1","id2"]).size().sort_index(level=["id1","id2"])

In [ ]:
#Save the informaton of connectedness
np.save("connectedness.npy", size_ids)
size_ids= pd.DataFrame(size_ids).reset_index()
size_ids.rename(columns={0:"N of connected events"}, inplace=True)
size_ids.to_csv("*.csv")